Create a CNN classifier to classify the pathology images as foreground vs background. The CNN should follow this architecture: CONV layer with 16 3x3 filters with pad 1 stride 1, RELU, POOL 2x2 with stride 2, CONV layer with 8 3x3 filters with pad 1 stride 1, RELU, POOL 2x2 with stride 2, Dense layer of size 64, RELU.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as nnF

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision
from torchvision import datasets, transforms, models
from torchviz import make_dot

from PIL import Image

In [28]:
dataset = datasets.ImageFolder('./Dataset/pathologyData/cancerData',
                 transform=transforms.ToTensor())

In [29]:
#Calculating the mean and standard deviation of images
loader = DataLoader(dataset,
                         batch_size=10,
                         num_workers=0,
                         shuffle=False)
mean = 0.
stdev = 0.
for images, _ in loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    stdev += images.std(2).sum(0)

mean /= len(loader.dataset)
stdev /= len(loader.dataset)

In [30]:
print(mean)
print(stdev)

tensor([0.8282, 0.8652, 0.8203])
tensor([0.1153, 0.0852, 0.0854])


In [31]:
my_transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.8282, 0.8652, 0.8203], std=[0.1153, 0.0852, 0.0854])
])

In [32]:
dataset = datasets.ImageFolder("./Dataset/pathologyData/cancerData", transform = my_transformations)

In [33]:
dataset_train, dataset_test = torch.utils.data.random_split(dataset, [int(len(dataset) * 0.75), int(len(dataset)*0.25)], generator=torch.Generator().manual_seed(42))

In [34]:
# Put into a Dataloader using torch library
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=30, shuffle=True)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=30, shuffle=False)

In [35]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        # nn.Conv2d API : torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1,padding=1)  # 16 filters of 3x3 size with pad =1 and stride = 1
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3,stride=1,padding=1) # 8 filters of 3x3 size with pad =1 and stride = 1
        # nn.Linear API : torch.nn.Linear(in_features, out_features, bias=True)
        self.fc1 = nn.Linear(25*25*8, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = nnF.max_pool2d(nnF.relu(self.conv1(x)), 2)
        x = nnF.max_pool2d(nnF.relu(self.conv2(x)), 2)
        x = x.flatten(start_dim=1)
        x = nnF.relu(self.fc1(x))
        x = nnF.log_softmax(self.fc2(x), dim=1)

        return x

model = CNN_Net()
print(model)


CNN_Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=5000, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)


In [36]:

# get a random training batch
iterator = iter(dataloader_train)
X_batch, y_batch = next(iterator)
print(X_batch.shape, y_batch.shape , model(X_batch).shape)

# pass a batch through the model and visualize the architecture
# NOTE: we do not have to explicitly call model.forward(inputs), instead we just do model(inputs)
# This is because PyTorch internally takes care of, giving us this syntactic sugar

#make_dot(model(X_batch), params=dict(model.named_parameters()))

torch.Size([30, 3, 100, 100]) torch.Size([30]) torch.Size([30, 2])


In [37]:
def training(model, device, data_loader, optimizer, criterion, epoch):
    model.train()
    loss_train = 0
    num_correct = 0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
        prediction = output.argmax(dim=1)
        num_correct += prediction.eq(target).sum().item()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.3f}%)]\tLoss: {:.3f}\tAccuracy: {:.3f}%'.format(
                epoch+1, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss_train / (batch_idx + 1),
                100. * num_correct / (len(data) * (batch_idx + 1))))
    loss_train /= len(data_loader)
    accuracy = num_correct / len(data_loader.dataset)
    return loss_train, accuracy
    

def testing(model, device, data_loader, criterion):
    model.eval()
    loss_test = 0
    num_correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            loss_test += loss.item()  # sum up batch loss
            prediction = output.argmax(dim=1)
            num_correct += prediction.eq(target).sum().item()
    loss_test /= len(data_loader)
    accuracy = num_correct / len(data_loader.dataset)
    return loss_test, accuracy

In [38]:
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')
model = CNN_Net().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [39]:
for epoch in range(7):
    loss_train, acc_train = training(model, device, dataloader_train, optimizer, criterion, epoch)
    print('Epoch {} Train: Loss: {:.3f}, Accuracy: {:.3f}%\n'.format(
        epoch+1, loss_train, 100. * acc_train))
    loss_test, acc_test = testing(model, device, dataloader_test, criterion)
    print('Epoch {} Test : Loss: {:.3f}, Accuracy: {:.3f}%\n'.format(
        epoch+1, loss_test, 100. * acc_test))

Train Epoch: 1 [0/2250 (0.000%)]	Loss: 0.673	Accuracy: 43.333%
Train Epoch: 1 [300/2250 (13.333%)]	Loss: 0.532	Accuracy: 67.273%
Train Epoch: 1 [600/2250 (26.667%)]	Loss: 0.457	Accuracy: 76.349%
Train Epoch: 1 [900/2250 (40.000%)]	Loss: 0.378	Accuracy: 83.333%
Train Epoch: 1 [1200/2250 (53.333%)]	Loss: 0.297	Accuracy: 87.317%
Train Epoch: 1 [1500/2250 (66.667%)]	Loss: 0.243	Accuracy: 89.739%
Train Epoch: 1 [1800/2250 (80.000%)]	Loss: 0.210	Accuracy: 91.311%
Train Epoch: 1 [2100/2250 (93.333%)]	Loss: 0.181	Accuracy: 92.488%
Epoch 1 Train: Loss: 0.171, Accuracy: 92.889%

Epoch 1 Test : Loss: 0.006, Accuracy: 99.867%

Train Epoch: 2 [0/2250 (0.000%)]	Loss: 0.004	Accuracy: 100.000%
Train Epoch: 2 [300/2250 (13.333%)]	Loss: 0.001	Accuracy: 100.000%
Train Epoch: 2 [600/2250 (26.667%)]	Loss: 0.004	Accuracy: 99.841%
Train Epoch: 2 [900/2250 (40.000%)]	Loss: 0.007	Accuracy: 99.677%
Train Epoch: 2 [1200/2250 (53.333%)]	Loss: 0.005	Accuracy: 99.756%
Train Epoch: 2 [1500/2250 (66.667%)]	Loss: 0.00